# STUDENT PERFORMANCES
## Machine Learning project
## Team: Amadou and Jamie

* Analysis of student performances
* Features: 33 
* Data: 649

There are 2 different csv files in the data folder. One (student-port.csv) being the data set of the students that took the portugese language course and another (student-mat.csv) of the students that are taking the math course. There are 382 students that are taking both courses. 

More information about the dataset is in the data folder, labeled student.txt

In [1]:
# Importing the libraries to be used:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.preprocessing import StandardScaler  # It is important in neural networks to scale the date
from sklearn.model_selection import train_test_split  # The standard - train/test to prevent overfitting and choose hyperparameters
from sklearn.metrics import accuracy_score # 
import matplotlib.pyplot as plt 

In [2]:
df1 = pd.read_csv("data\student-por.csv",sep=";") #read file from different directory
df2 = pd.read_csv("data\student-mat.csv", sep = ";")
print(df1)
#df1.to_csv("port.csv") # made in to a csv after being made in to a table - easier to read compared to original excel file


    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
644     MS   F   19       R     GT3       T     2     3  services     other   
645     MS   F   18       U     LE3       T     3     1   teacher  services   
646     MS   F   18       U     GT3       T     1     1     other     other   
647     MS   M   17       U     LE3       T     3     1  services  services   
648     MS   M   18       R     LE3       T     3     2  services     other   

     ... famrel freetime  goout  Dalc  Walc health 

In [3]:
#Splitting the data accordingly
X = df1.iloc[:, :30]
y = df1.iloc[:, 32:]
X.dropna()
y.dropna()
y = np.array(y) #changed to array to be easier to work with

## Converting the specific features to binary to make it easier to work with
### school:  GP = 1 , MS = 0
### sex:  F = 1, M = 0
### address: U(urban) = 1, R(rural) = 0
### family size:  LE3 = 1, GT3 = 0
### pstatus: T(living together) = 1, A(living apart) = 0

In [4]:
#print(X.iloc[:, 0])
#print(X.iloc[1, 0])
switch = {0: ("GP", "MS"), 1: ("F", "M"), 3: ("U", "R"), 4: ("LE3", "GT3"), 5:("T", "A"), 15:("yes","no"), 16:("yes","no"),
         17:("yes","no"), 18:("yes","no"), 19:("yes","no"), 20:("yes","no"), 21:("yes","no"), 22:("yes","no")}
#ignore = ["age", "Medu", "Fedu","Mjob", "Fjob", 'reason', 'guardian','traveltime', 'studytime',
#       'failures']
#print(X.columns)
#print(X.iloc[0,:])
for column in range(len(X.columns)):
    if column not in switch:
        continue
    for i in range(len(X.iloc[:,column])):
       #print(X.iloc[i, column] == "U",X.iloc[i, column], switch[column])
        if X.iloc[i,column] == switch[column][0]:
            #print(X.iloc[i, column], switch[column][0])
            X.iloc[i,column] = 1
        elif X.iloc[i, column] == switch[column][1]:
            X.iloc[i,column] = 0
X.drop(["Mjob", "Fjob", "reason", "guardian"], axis = 1, inplace=True) #Remove the ones with more than 1 and 0 answers
print(X.shape)

(649, 26)


In [5]:
X_scale = StandardScaler()
X = np.array(X_scale.fit_transform(X))#changed to array to be easier to work with

In [6]:
#Split the data into training and test set.  60% training and %40 test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# FROM here on out the markdown split up which model. Add cells according to which mark down. Here is the order
## 1. Linear Regression
## 2. Neutral Network 
## 3. SVM

# Linear Regression

# Neutral Network
## some of these code are taken from hw6 as I deemed them essential 

In [7]:
#Code copied from hw6, but edited to match our y values
def convert_y_to_vect(y):
    y_vect = np.zeros((len(y), 20))
    for i in range(len(y)):
        y_vect[i, y[i]] = 1
    return y_vect

In [8]:
y_v_train = convert_y_to_vect(y_train)
y_v_test = convert_y_to_vect(y_test)
print("Testing training set shape: ",y_v_train.shape)
print("Testing testing set shape: ",y_v_test.shape)

Testing training set shape:  (389, 20)
Testing testing set shape:  (260, 20)


In [9]:
# Sigmoid activation function - depends on whether we want to use it or not
def f(z):
    return 1 / (1 + np.exp(-z))


def f_deriv(z):
    return f(z) * (1 - f(z))

In [10]:
def calculate_out_layer_delta(y, a_out, z_out):
    # delta^(nl) = -(y_i - a_i^(nl)) * f'(z_i^(nl))
    return -(y-a_out) * f_deriv(z_out) 


def calculate_hidden_delta(delta_plus_1, w_l, z_l):
    # delta^(l) = (transpose(W^(l)) * delta^(l+1)) * f'(z^(l))
    return np.dot(np.transpose(w_l), delta_plus_1) * f_deriv(z_l)